In [ ]:
import numpy as np
import xarray as xr
from scipy import ndimage
from scipy.spatial import ConvexHull
import pandas as pd
import pickle
import cftime
import multiprocessing as mp

## Compute heat transport like from observations
We co-locate eddies from tracks with regions in eddymask. Then we compute heat content as a sum over all the T' in one connected region and relate this to the translational eddy velocity computed from the latitudinal displacement of the eddy between `t-1` and `t+1`.

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

We run the calculations in parallel with `multiprocessing`, here we define how many cpus to use.

In [ ]:
ncpu = 32

Define a function to get indeces of the regions detected.

In [ ]:
def get_indeces(data):
    d = data.ravel()
    f = lambda x: np.unravel_index(x.index, data.shape)
    return pd.Series(d).groupby(d).apply(f)

Define the function that computes the eddy velocities as described above

In [ ]:
def calc_eddy_vel(r):
    d = dummy.copy()
    # loop over the assigned tracks
    for rt in rangeTracks[r]:
        track = tracks[rt]
        lenTrack = len(track["time"])
        # for each track, compute velocity at each time
        for time in np.arange(0, lenTrack):
            with dask.config.set(**{'array.slicing.split_large_chunks': False}):
                # get the latitude and longitude of the eddy
                lon = np.mean(track["eddy_i"][time])
                lat = np.mean(track["eddy_j"][time])
                # load the eddymask of this time step
                if ((track["time"][time] <= lastTime)
                    & (track["time"][time] >= firstTime)):
                    emt = em.sel(time=track["time"][time]).copy()
                else:
                    break
                # define connected regions
                regions, nregions = ndimage.label((emt > 0.).astype(int))
                region_index = get_indeces(regions)
                point = np.array([lon, lat])
                i = 0
                is_inside = False
                # check which region the current eddy is associated with
                while not is_inside: 
                    if i < (nregions - 1):
                        i += 1
                    else:
                        break
                    p = np.array([region_index[i][1], region_index[i][0]]).T
                    try:
                        hullp = ConvexHull(p)
                        poly = plt.Polygon(p[hullp.vertices, :])
                        is_inside = poly.contains_point(point)
                    except:
                        continue
                # if the region is found, compute the velocity and multiply the mask by it
                if is_inside:
                    # the first and last velocities are only based on the first and last time
                    if time == 0:
                        lat1 = track["lat"][time]
                        lat2 = track["lat"][time + 1]
                        vobs = (lat2 - lat1) / (3600 * 24 * 5)
                    elif ((time > 0) 
                          & (time < (lenTrack - 1))):
                        lat1 = track["lat"][time - 1]
                        lat2 = track["lat"][time + 1]
                        vobs = (lat2 - lat1) / (3600 * 24 * 10)
                    elif time == (lenTrack - 1):
                        lat1 = track["lat"][time - 1]
                        lat2 = track["lat"][time]
                        vobs = (lat2 - lat1) / (3600 * 24 * 5)
                    # use the correct time (corrsponding to the on in d)
                    if ((track["time"][time] <= lastTime)
                        & (track["time"][time] >= firstTime)):
                        realTime = (d.time == track["time"][time]).argmax()
                        d[realTime,:,:] = (d[realTime,:,:] * vobs).where(regions == i, 
                                                                           other=d[realTime,:,:])
                    else:
                        break
    return d

Now we do the calculation of velocities for each year, multiply by $\rho$, $C_{p}$ and $T'$ to get the heat transport.

In [ ]:
# open the eddy tracks
with open(eddypath + 'tracks_02010101_03001230.pickle', 'rb') as f:
    tracks = pickle.load(f)

f.close()
# open the data interpolated to F-points (eddies are defined on F-points)
data_int = xr.open_mfdataset(eddypath + 'interp_data.extend800.0*.nc', 
                             concat_dim="time", combine="nested",
                             data_vars='minimal', coords='minimal', 
                             compat='override').isel(lon=slice(0, 240))
# load the eddy mask
eddymask = xr.open_mfdataset(eddypath + 'eddymask_binary_0201-0300_k15.new0.3.all.nc', 
                             concat_dim="time", combine="nested",
                             data_vars='minimal', coords='minimal', 
                             compat='override').eddymask_binary.mean("z")
rho = 1035.
Cp = 3994.
t1 = np.arange(201, 300, 10)
t2 = np.arange(210, 301, 10)
# loop over decades
for a, b in zip(t1, t2):
    y1 = "0" + str(a)
    y2 = "0" + str(b)  
    # compute T' for the decade
    di = data_int.sel(time=slice(y1 + '-01-01', y2 + '-12-30'))
    tbar = di.THETA.groupby('time.month').mean("time")
    tprime = di.THETA.groupby('time.month') - tbar
    tprime = tprime.compute()
    # loop over years to calculate translational velocities
    for y in np.arange(a, b+1):
        result = 0
        Y = "0" + str(y)
        # select variables and timestamp for this year
        tp = tprime.sel(time=slice(Y + '-01-01', Y + '-12-30')).compute()
        savetime = tp.time.isel(time=slice(int(len(tp.time) / 2), int(len(tp.time) / 2) + 1))
        em = (eddymask.sel(time=slice(Y + '-01-01', Y + '-12-30')) * (tp / tp)).compute()
        # create dummy to write into
        dummy = em.copy()
        # detect first and last timestamps
        firstTime = dummy.time[0].values
        lastTime = dummy.time[-1].values
        # find out which tracks correspond to these timestamps
        startTrack = 0
        while tracks[startTrack]["time"][-1] < firstTime:
            startTrack += 1   
        lastTrack = 0
        while ((tracks[lastTrack]["time"][0] < lastTime)
               & (lastTrack < len(tracks) - 1)):
            lastTrack += 1  
        lastTrack -= 1
        rT = np.arange(startTrack, lastTrack)
        # divide the range of tracks into equally long segments
        lenSeg = np.floor(len(rT) / ncpu).astype(int)
        rangeTracks = {}
        for n in np.arange(0, ncpu):
            start = n * (lenSeg)
            end = (n + 1) * lenSeg
            if n == ncpu - 1:
                end = lastTrack
            rangeTracks[n] = rT[start:end]
        # compute eddy velocity in parallel for each segment
        with mp.Pool(ncpu) as pool:
            result = pool.map(calc_eddy_vel, np.arange(0, len(rangeTracks)))
        # comine the results into one array
        final = 0
        for i in np.arange(0, len(rangeTracks)):
            final += result[i].where(result[i]!=1, other=0)
        eddyV = final.compute()
        # compute THT from eddyV and T' and write to disk
        THTobs = xr.DataArray(
            (rho * Cp * (eddyV * tp) * di.dxV).sum("lon").mean("time").values[None, :, :], 
            coords={"time": savetime, "z": tp.z, "lat": tp.lat},
            dims=["time", "lat", "z"], name='THTobs')
        THTobs.to_netcdf(path + "post/THTobs." + Y + ".nc")
        for s in ["DJF", "MAM", "JJA", "SON"]:
            THTobsSeason = xr.DataArray(
                (rho * Cp * (eddyV * tp) 
                 * di.dxV).sum("lon").groupby("time.season").mean("time").sel(season=s).values[None, :, :],
                coords={"time": savetime, "z": tp.z, "lat": tp.lat},
                dims=["time", "lat", "z"], name='THTobs')
            THTobsSeason.to_netcdf(path + "post/THTobs." + s + "." + Y + ".nc")
        # create trackmask (equivalent to eddymask but only for tracked eddies)
        tm = eddyV.where(eddyV==0, other=1).isel(z=15)
        tm.rename("trackmask").to_netcdf(eddypath + "trackmask_" + Y + ".nc")


### repeat for anticyclones and cyclones